In [52]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import arrow
import re
from matplotlib import pyplot as plt
import plotly.express as px
import plotly
import plotly.graph_objects as go
import kaleido

In [53]:
def preprocessing(path):
    
    frmt = 'MM_DD_YYYY__HH-mm-ss_A'
    arr = []
    qwe = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
           'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 
           'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
    
    files = [f for f in os.listdir(path) if os.path.isfile(path + '//' + f)]
    
    for file in files:

        to_append = arrow.get(file.replace(file[:3], qwe[file[:3]]).split('.')[0], [frmt, 'MM/DD/YYYY HH:mm:ss']).isoformat()
        arr.append(to_append.split('T')[1].split('+')[0])

    arr_sorted = sorted(arr, key=lambda x: datetime.strptime(x, '%H:%M:%S'), reverse=False)
    
    arr_dir_sorted = [x for _,x in sorted(zip(arr,files), key=lambda x: datetime.strptime(x[0], '%H:%M:%S'), reverse=False)]
    
    return arr_dir_sorted


def process_file(path):
    
    file = open(path, 'r')
    string_arr = file.readlines()[22:]
    nums = list(map(lambda x: re.split(',', re.sub('[ \n]', '', x))[1], string_arr))
    
    return list(map(float, nums))


def make_df(filename_array, path):
    
    to_df = []
    k = 0
    for file in filename_array:
        to_df.append(process_file(path + '/' + file))
        k+=1
    
    df = pd.DataFrame(to_df)
    df.columns = df.columns + 1
    return df

def plot_graphs(df, path):
    
    os.makedirs('{}/Processed'.format(path, exist_ok = True))
    for i in range(65):

        fig, ax=plt.subplots(figsize = (6,6))
        plt.plot(df.columns, df.iloc[i])
        plt.title('M/Z = {}'.format(i+1))
        plt.xlabel('Measurement number')
        plt.ylabel('Response')
        plt.savefig('{}//Processed//{}.png'.format(path,i+1), dpi = 600)
        plt.close(fig)
        
def plot_interactive_graph(df, mz):
    nd = 4
    fig = px.line(x = np.arange(df.shape[0]-nd)+1, y = df.loc[nd:, mz], 
              labels = {'x': 'Номер измерения', 'y': 'Сигнал'}, title = 'M/Z = {}'.format(mz))
    fig.update_layout(yaxis = dict(showexponent = 'all',exponentformat = 'e'))
    return fig

In [54]:
root_path = os.path.join(os.getcwd(), os.pardir)
path = os.path.join(root_path, 'data\\28_06_2024\\base\\')
output_path = os.path.join(root_path, 'output\\')

dirs_sorted_list = preprocessing(path)
processed = make_df(dirs_sorted_list, path)

In [55]:
processed

,1,2,3,4,5,6,7,8,9,10,...,56,57,58,59,60,61,62,63,64,65
0,2.320000e-08,8.690000e-08,3.830000e-09,5.040000e-10,2.310000e-10,2.440000e-10,2.310000e-10,2.450000e-10,1.390000e-10,1.410000e-10,...,1.760000e-10,4.110000e-10,7.970000e-10,8.480000e-12,2.180000e-10,-4.670000e-11,7.740000e-11,0.000000e+00,3.200000e-10,2.040000e-10
1,1.490000e-08,6.960000e-08,3.350000e-09,4.010000e-10,2.060000e-10,2.780000e-10,2.050000e-10,-5.300000e-11,1.110000e-10,1.910000e-11,...,2.510000e-10,5.470000e-10,4.060000e-10,2.410000e-10,2.550000e-11,-3.820000e-11,2.270000e-10,1.940000e-10,2.880000e-10,3.030000e-10
2,1.590000e-08,6.210000e-08,3.380000e-09,3.700000e-10,1.590000e-10,1.050000e-10,1.540000e-10,1.990000e-10,1.560000e-10,1.590000e-10,...,1.910000e-10,3.120000e-10,4.190000e-10,-2.760000e-11,9.540000e-11,1.610000e-10,1.260000e-10,2.140000e-10,3.550000e-10,3.540000e-10
3,1.450000e-08,5.660000e-08,3.020000e-09,1.950000e-10,2.440000e-11,-5.510000e-11,-5.940000e-11,9.440000e-11,-1.030000e-10,-1.060000e-11,...,1.660000e-10,1.610000e-10,2.320000e-10,1.560000e-10,5.510000e-11,-9.330000e-11,-2.760000e-11,9.010000e-11,3.820000e-11,1.590000e-11
4,1.150000e-08,5.350000e-08,3.150000e-09,2.400000e-10,1.160000e-10,1.360000e-10,-5.730000e-11,1.050000e-10,-2.650000e-11,-1.030000e-10,...,8.170000e-11,2.440000e-10,1.120000e-10,1.560000e-10,-9.540000e-12,-3.080000e-11,6.150000e-11,-7.100000e-11,1.270000e-10,1.800000e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2.740000e-09,1.750000e-08,1.730000e-09,3.180000e-10,2.550000e-11,1.510000e-10,-3.080000e-11,1.220000e-10,-4.980000e-11,8.170000e-11,...,1.860000e-10,-9.540000e-12,3.920000e-11,1.510000e-10,2.550000e-10,-1.170000e-11,2.740000e-10,3.180000e-12,9.970000e-11,-1.380000e-11
116,2.540000e-09,1.740000e-08,1.970000e-09,2.250000e-10,1.130000e-10,1.020000e-10,2.320000e-10,3.090000e-10,2.640000e-10,2.050000e-10,...,2.520000e-10,9.650000e-11,1.090000e-10,1.980000e-10,-4.140000e-11,4.240000e-11,1.040000e-10,1.660000e-10,1.340000e-10,3.400000e-10
117,2.690000e-09,1.760000e-08,1.620000e-09,-6.360000e-12,9.230000e-11,1.620000e-10,1.630000e-10,1.530000e-10,9.120000e-11,2.120000e-12,...,-5.200000e-11,8.800000e-11,-5.940000e-11,6.890000e-11,1.860000e-10,1.400000e-10,9.010000e-11,3.500000e-11,1.270000e-10,-4.880000e-11
118,2.690000e-09,1.760000e-08,1.760000e-09,2.840000e-10,2.180000e-10,2.970000e-11,4.980000e-11,5.940000e-11,5.620000e-11,7.210000e-11,...,-3.710000e-11,2.170000e-10,2.230000e-11,9.010000e-11,1.110000e-10,0.000000e+00,5.300000e-11,1.080000e-10,9.440000e-11,7.420000e-12


In [56]:
output_filename_no_extension = os.path.join(output_path, os.path.basename(os.path.dirname(path)))
processed.to_csv(output_filename_no_extension + ".csv", sep = ";", decimal = ",")

In [57]:
mz = 44# b это m/z
plot_interactive_graph(processed, mz)

In [58]:
fig = go.Figure()
nd = 4
mz = 28
fig.add_trace(go.Scatter(x = np.arange(processed.shape[0]-nd)+1, y = processed.loc[nd:, mz], mode = "lines", name="Trace #1"))
fig.add_trace(go.Scatter(x = np.arange(processed.shape[0]-nd)+1, y = processed.loc[nd:, mz], mode = "lines", name="Trace #2"))

fig.show()

In [59]:
fig = plot_interactive_graph(processed, mz)

In [60]:
fig.write_image(output_filename_no_extension + ".png", format="png", height = 1500, width = 1000, engine = "kaleido")